In [32]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Read the CSV file
flights_df = pd.read_csv('./2008.csv')

flights_df = flights_df.dropna(subset=['Year', 'Month', 'DayofMonth', 'DepTime', 'ArrTime', 'Origin', 'Dest'])

flights_df.rename(columns={'Year': 'year', 'Month': 'month', 'DayofMonth': 'day'}, inplace=True)

# Construct FL_DATE from Year, Month, and DayofMonth
flights_df['FL_DATE'] = pd.to_datetime(flights_df[['year', 'month', 'day']])

# Extract only the relevant columns
flights_df = flights_df[['FL_DATE', 'DepTime', 'ArrTime', 'Origin', 'Dest']]

# Extract hours only for DepTime and ArrTime
flights_df['DEP_HOUR'] = (flights_df['DepTime'] // 100).fillna(0).astype(int)
flights_df['ARR_HOUR'] = (flights_df['ArrTime'] // 100).fillna(0).astype(int)

# Estimate airplanes directly from total flights
def estimate_airplanes(flights_df, H):
    """
    Estimate the total number of airplanes based on the total number of flights.
    Assume that a certain percentage of flights require dedicated airplanes.
    """
    total_flights = len(flights_df)
    # Assume each airplane can handle 5 flights in a day on average
    flights_per_airplane = 4.5
    airplanes_needed = max(1, int(total_flights / flights_per_airplane))
    return airplanes_needed

# Assign start airplanes proportionally to departing flights
def assign_start_airplanes(departing_airports, departing_counts, airplanes_needed):
    """
    Assign airplanes to each departing airport proportional to the number of departures.
    """
    airport_start_planes = {}
    total_departures = sum(departing_counts.values())

    # Allocate airplanes proportionally to departure counts
    for airport, count in departing_counts.items():
        airport_start_planes[airport] = max(1, int((count / total_departures) * airplanes_needed))

    # Distribute any remaining airplanes randomly among the airports
    remaining_airplanes = airplanes_needed - sum(airport_start_planes.values())
    if remaining_airplanes > 0:
        for airport in random.sample(list(departing_airports), len(departing_airports)):
            if remaining_airplanes > 0:
                airport_start_planes[airport] += 1
                remaining_airplanes -= 1
            else:
                break

    return airport_start_planes

# Generate simulated data for a single horizon
def generate_simulated_data(flights_df, H, airplanes_needed, output_file):
    # Randomly pick a starting point in the dataset (use FL_DATE instead of indices)
    start_date = flights_df['FL_DATE'].min() + timedelta(days=random.randint(0, 365 - H))
    end_date = start_date + timedelta(days=H)

    # Filter flights within the horizon
    flights_in_horizon = flights_df[
        (flights_df['FL_DATE'] >= start_date) &
        (flights_df['FL_DATE'] < end_date)
    ]

    # Get the set of departing airports and their counts for the start day
    flights_start_day = flights_in_horizon[flights_in_horizon['FL_DATE'] == start_date]
    departing_airports = flights_start_day['Origin'].unique()
    departing_counts = flights_start_day['Origin'].value_counts().to_dict()

    # Assign airplanes to departing airports
    airport_start_planes = assign_start_airplanes(departing_airports, departing_counts, airplanes_needed)

    # Prepare output data for flights
    flights_data = []
    for _, flight in flights_in_horizon.iterrows():
        dep_day = (flight['FL_DATE'] - start_date).days
        arr_day = dep_day + (flight['ARR_HOUR'] < flight['DEP_HOUR'])  # Handle day rollover

        # Skip flights where departure and arrival are in the same hour
        if dep_day == arr_day and flight['DEP_HOUR'] == flight['ARR_HOUR']:
            continue

        flights_data.append(
            f"{flight['Origin']} {dep_day} {flight['DEP_HOUR']} {flight['Dest']} {arr_day} {flight['ARR_HOUR']}"
        )

    # Prepare airplane starting data (only airports with assigned planes)
    airplanes_data = [
        f"{airport} {airport_start_planes[airport]}"
        for airport in airport_start_planes if airport_start_planes[airport] > 0
    ]

    # Write to file
    with open(output_file, 'w') as f:
        f.write(f"{len(flights_data)} {len(airplanes_data)}\n")
        f.writelines("\n".join(flights_data) + "\n")
        f.writelines("\n".join(airplanes_data) + "\n")

    # Return stats for departing airports
    return departing_counts

# Main execution
if __name__ == "__main__":
    horizons = [2, 3, 5, 7, 10, 15, 30]
    all_stats = {}

    for H in horizons:
        # Estimate airplanes for the current horizon
        airplanes_needed = estimate_airplanes(flights_df, H)

        # Generate simulated data for the current horizon
        output_file = f"../Inputs/data_{H}.txt"
        departing_stats = generate_simulated_data(flights_df, H, airplanes_needed, output_file)

        print(f"Simulated data for horizon {H} days written to {output_file}.")
        print(f"Estimated airplanes needed for H={H}: {airplanes_needed}")

        # Save stats for departing airports
        all_stats[H] = departing_stats

    # Print summary of departing airports for each horizon
    print("\nSummary of departing airport statistics per horizon:")
    for H, stats in all_stats.items():
        print(f"Horizon {H} days:")
        print(pd.DataFrame(list(stats.items()), columns=['Airport', 'Departures']).sort_values(by='Departures', ascending=False))


Simulated data for horizon 2 days written to ../Inputs/data_2.txt.
Estimated airplanes needed for H=2: 1524017
Simulated data for horizon 3 days written to ../Inputs/data_3.txt.
Estimated airplanes needed for H=3: 1524017
Simulated data for horizon 5 days written to ../Inputs/data_5.txt.
Estimated airplanes needed for H=5: 1524017
Simulated data for horizon 7 days written to ../Inputs/data_7.txt.
Estimated airplanes needed for H=7: 1524017
Simulated data for horizon 10 days written to ../Inputs/data_10.txt.
Estimated airplanes needed for H=10: 1524017
Simulated data for horizon 15 days written to ../Inputs/data_15.txt.
Estimated airplanes needed for H=15: 1524017
Simulated data for horizon 30 days written to ../Inputs/data_30.txt.
Estimated airplanes needed for H=30: 1524017

Summary of departing airport statistics per horizon:
Horizon 2 days:
    Airport  Departures
0       ATL        1199
1       ORD        1000
2       DFW         813
3       DEN         704
4       LAX         644
